# Aufbau der Übungen

Für die Vermittlung der Übungen verwenden wir einfache Jupyter Notebooks. Um die Übungen darzustellen und um sie abarbeiten zu können, muss der `exercise_server` container laufen. Dieser `exercise_server` wird aber nur dazu verwendet, durch die Übungen zu führen und Vorschläge für Musterlösungen anzuzeigen. Die Ausführung der Übungen erfolgt ausserhalb des `exercise_servers` in separaten Containern, die schrittweise eingeführt und aufgebaut werden.

Einige Übungen kommen als Fragen daher, die direkt im Jupyter Notebook beantwortet werden können. Die Antwort kann anschliessend mit der Musterlösung verglichen werden.

Für die meisten Übungen muss aber beispielsweise ein Docker Container konfiguriert oder Code geschrieben werden.

# Übung 1: Basis-Infrastruktur

Zu Beginn des Workshops sieht die Infrastruktur wie in folgendem Diagramm gezeigt aus.

![overview.png](overview.png)

## Service Definition

Wir beginnen demnach mit zwei getrennten Docker-Netzwerken und drei Containern:
 * Dem Exercise Container, welchen du gerade verwendest
 * Dem Object Store
 * Der Entwicklungsumgebung in Form eines JupyterLab Servers

Schau Dir unter den Pfaden
* `exercises/`
* `exercises/containers/development_env/`
* `exercises/containers/minio.yml`

die Dockerfiles und docker-compose (*.yml) Files der development environment und von minio an, um den Einstieg in die Workshop-Infrastruktur zu finden. Die weiteren Dockerfiles und compose Files kannst Du vorerst ignorieren. Wie ist die Hierarchie der Docker Files aufgebaut?

Im top-level compose file `docker-compose.yml` werden via Include die zwei Netzwerke definiert. Danach werden die  compose Files der einzelnen Services gelistet. Die Services sind dann im Unterordner `containers` definiert. Dort wo notwendig (d.h. wenn es ein `Dockerfile` benötigt, z.B. bei der `development_env`) in einem eigenen Verzeichnis, oder dann direkt als YAML File ohne Verzeichnis (z.B. bei `minio.yml`)

<div class="alert alert-block alert-info">
<b>Achtung: </b>Einige Dienste sind noch auskommentiert. Sie werden noch nicht benötigt, wir werden sie später schrittweise aktivieren.</div>

----

Starte nun alle container. Dies dauert beim ersten mal einen Moment, da die Images geladen werden müssen.

Dazu führt Du im Verzeichnis `/exercises`, wo sich das top-level compose file `docker-compose.yml` befindet, den Befehl

    docker compose up

aus. Der Prozess bleibt so im Vordergrund und Du kanst das Terminal Window offen lassen. Später kannst Du den Prozess mit `ctrl-c` einfach stoppen.

Du kannst aber auch `docker compose up -d` ausführen, um den Prozess in den Hintergrund zu schicken.

----

## Docker-Netzwerke

Wir simulieren eine Trennung von Entwicklungs-Umgebung und produktiver Umgebung, indem wir zwei Docker-Netzwerke definieren und jedem Container entweder dem einem oder dem anderen der beiden Netzwerke zuordnen. Nur Dienste, welche produktiv laufen, und auch aus der Entwicklungsumgebung erreichbar sein müssen, werden beiden Netzwerken zugeordnet.

Nenne einige solche Dienste, welche typischerweise sowohl dem Development- wie auch dem Production Netzwerk zugeordnet sein sollten.

Einige mögliche Dienste, die aus beiden Netzwerken zugreifbar sein müssen, sind

* Object Store
* Gitlab
* Docker Registry
* Model Registry

----

## Object Store

Wir verwenden für die Datenablage einen Object Store. In Object Store speichern wir Trainingsdaten, Artefakte aus dem Trainingsprozess, Features und Predictions. Eine lokale [MinIO](https://min.io/) Instanz mit minimaler Konfiguration genügt uns hierzu. 

Schau Dir das Docker Compose File des Object Stores an. Welches lokale Verzeichnis wird für die Ablage der Object Store Daten verwendet?

Im docker-compose File von minio (`containers/minio.yml`) kannst Du den volume mount finden:

```
    volumes:
      - ../s3_data:/data
```

Daran siehst Du, dass die Daten ausserhalb des Containers im lokalen Verzeichnis `s3_data` abgelegt sind. Innerhalb des Containers liegen die Daten im Verzeichnis `/data`. In der command direktive sieht man, dass dem minio daemon das Verzeichnis `/data` beim Start als Argument angegeben wird:

```
command: server --address "0.0.0.0:9000" /data
```

----

# Development Environment

Wir simulieren die Entwicklungsumgebung, in welcher Data Scientists Analysen durchführen und Modelle entwickeln, mit einem einzelnen Container, welcher eine Jupyter Umgebung zur Verfügung stellt. Im Rahmen des Workshops reicht dies vollkommen aus. In der Realität wäre eine Entwicklungsumgebung natürlich etwas komplexer, mindestens eine IDE und GPUs müssten neben einer Jupyter-Umgebung auch noch zur Verfügung stehen.

Schau Dir das docker-compose File der Entwicklungsumgebung an, um die URL und den Port zu finden, über den du auf Jupyterlab zugreifst.

Im File `development_env/development_env.yml` findest Du den Abschnitt 

```
      ports:
         - 127.0.0.1:8080:8888
```

Der Port, unter welchem Jupyterlab im Container läuft, ist 8888. Er wird auf den Port 8080 ausserhalb des Containers gemappt. Du kannst also via die URL [http://localhost:8080](http://localhost:8080) auf die Entwicklungsumgebung zugreifen.

----

Greife via Browser auf die Entwicklungsumgebung zu. Du kannst direkt einloggen, ohne ein Token oder andere Credentials angeben zu müssen. Das wurde aus Bequemlichkeitsgründen für den Workshop so konfiguriert. In einer richtigen Installation sollte natürlich eine Autorisierung und Authentifizierung aktiviert werden.

Versuche herauszufinden, wie bzw. wo die Autorisierung für Jupyter deaktiviert wurde.

Du findest die Konfiguration direkt im Dockerfile:

```
# generate a config directory and the default config
RUN jupyter notebook --generate-config

# configure jupyter for easy access (don't do this in production!!!)
RUN echo "c.NotebookApp.open_browser = False" >> /root/.jupyter/jupyter_notebook_config.py && \
    echo "c.JupyterApp.answer_yes = True" >> /root/.jupyter/jupyter_notebook_config.py && \
    echo "c.NotebookApp.ip = '*'" >> /root/.jupyter/jupyter_notebook_config.py && \
    echo "c.NotebookApp.allow_origin = '*'" >> /root/.jupyter/jupyter_notebook_config.py && \
    echo "c.NotebookApp.token = ''" >> /root/.jupyter/jupyter_notebook_config.py && \
    echo "c.NotebookApp.password = ''" >> /root/.jupyter/jupyter_notebook_config.py
```

----

Greife nun mit Deinem Browser auf die Entwicklungsumgebung zu. Du solltest folgendes sehen:

![jupyter.png](jupyter.png)

Du kannst auch ein Jupyterlab öffnen, wenn du das lieber verwendest:

![jupyter_open_lab.png](jupyter_open_lab.png)

![jupyterlab.png](jupyterlab.png)

Halten wir noch einmal den Unterschied zwischen den beiden Jupyter Servern fest:
 * Der `exercise_server` dient dazu, durch die Übungen zu führen. In ihm siehst du den Übungsablauf und allfällige Lösungsvorschläge. Du greifst via Port 8888 auf den Exercise Server zu.
 * Die `Development Environment` ist Teil der Infrastruktur, die wir im Workshop nach und nach aufbauen bzw. konfigurieren. Sie dient dazu, die Umgebung zu simulieren, in welcher Data Scientists Analysen durchführen und Machine Learning Modelle bauen und trainieren.

Verwende nun die Entwicklungsumgebung, um ein neues Jupyter Notebook zu erstellen. Nenne es `hello_world.ipynb` und lasse es ein Hello World ausgeben.

Das kriegst Du auch ohne Musterlösung hin ;-)

----

# Lesen und Schreiben aus dem Object Store

Wir verwenden `s3fs`, um Files in den Object Store zu schreiben und aus ihm zu lesen. Du findest die Dokumentation [hier](https://s3fs.readthedocs.io/en/latest/).

Öffne nun ein neues Notebook in der Development Environment (oder recycle das vorherige helloworld Notebook) um ein wenig Übung im Umgang mit dem Object Store zu bekommen:
 * Erstelle einen bucket
 * Schreibe ein Textfile in diesen bucket
 * Zeige den Inhalt des Buckets an
 * Probiere einige weitere Befehle des [APIs](https://s3fs.readthedocs.io/en/latest/api.html) aus

Hier ist ein Codebeispiel. Beachte jedoch, dass du diese Codezelle  nicht direkt aus diesem Notebook heraus ausführen kannst.

In [ ]:
import s3fs

# create a reference to the object store
s3 = s3fs.S3FileSystem()

# create a bucket
s3.mkdir("mybucket")

# write a file. According to the documentation, only binary mode works, although writing a string directly seems to work as well
with s3.open('mybucket/hello.txt', 'wb') as f:
    f.write(r'Hello World'.encode('utf-8'))

# some more commands
s3.ls('mybucket')
s3.du('mybucket/hello.txt')

----

Kannst Du erklären, wie die Autorisierung und die Verbindung funktioniert?

Im docker-compose File der Entwicklungsumgebung siehst du, dass die folgenden Umgebungsvariablen gesetzt werden:

```
      environment:
        - FSSPEC_S3_KEY=${MINIO_ADMIN}
        - FSSPEC_S3_SECRET=${MINIO_SECRET}
        - FSSPEC_S3_ENDPOINT_URL=${MINIO_ENDPOINT}
```

Durchsucht man die s3fs Dokumentation nach der ersten dieser drei Variablen, [findet man](https://s3fs.readthedocs.io/en/latest/index.html?highlight=FSSPEC_S3_KEY#s3-compatible-storage), dass diese Variablen, wenn vorhanden, beim Erstellen des s3 Objectes mit `s3fs.S3FileSystem()` verwendet werden, um die Credentials und den Endpoint zu setzen. Deshalb musstest du bei der obigen Übung diese Infos nicht explizit angeben.

Wenn du nun das docker-compose File des Object Stores anschaust, siehst du dass dem Server beim Start gesagt wird, er solle auf Port 9000 hören und via Umgebungsvariablen Username und Passwort des root Users mitgeteilt werden. Wir verwenden hier der Einfachheit halber für alles diesen User, was in der Praxis natürlich nicht gemacht wird.


```
    command: server --address "0.0.0.0:9000" /data
    hostname: objectstore
    expose:
      - "9000"
    environment:
      - MINIO_ROOT_USER=${MINIO_ADMIN}
      - MINIO_ROOT_PASSWORD=${MINIO_SECRET}
    volumes:
      - ../s3_data:/data
    networks:
      - development
      - production
```

----

Und wo werden Usernamen, Secret und Endpoint effektiv gesetzt?

Die effektiven Werte für Passwort und Endpoint werden in einem .env File gesetzt:

```
$ cat .env
MINIO_ADMIN=xxx
MINIO_SECRET=xxx
MINIO_ENDPOINT=http://objectstore:9000
```

Docker-compose [verwendet dieses .env File](https://docs.docker.com/compose/environment-variables/set-environment-variables/#use-the-env_file-attribute), um die Platzhalter entsprechend zu befüllen.

----

## Zusammenfassung

Wir starten also mit der folgenden Infrastruktur:

![summary.png](summary.png)

Der Zweck des Exercise Servers ist, durch die Übungen zu führen. Er ist nich mit dem Rest der Infrastruktur verbunden, in den Notebooks des Exercise Servers kann kein Code ausgeführt werden.

Die Umsetzung der Übungen erfolgt in Containern, welche den Docker Netzwerken `Development` oder `Production` zugeordnet sind.

Bisher haben wir den Object Store, welcher aus beiden beiden Netzwerken verwendbar ist und die Entwicklungsumgebung, die nur auf das  Enticklungsnetzwerk Zugriff hat.

Im Laufe des Workshops werden weitere Dienste hinzukommen.

Damit ist die erste Übung abgeschlossen. **Bitte quittiere auf [Mentimeter](https://www.menti.com/alaxbnek73eu), dass du mit der Übung durch bist**, damit der Dozent eine Idee hat, wieviele Teilnehmer die Übung abgeschlossen haben.

Du kannst nun gleich mit der nächsten Übung weiterfahren: `07_Ein_einfaches_Modell_Code.ipynb`